In [ ]:
# 01_seed_static.py
# -------------------------------------------------------------------
# Full static seed with hard reset, robust sequence reset, and seeding:
# - Wipes all business tables (children -> parents)
# - Resets Oracle identity/classic sequences (XE-safe)
# - Seeds BRANCHES, MANAGERS, CAR_CATEGORIES, IOT_DEVICES, CARS
# -------------------------------------------------------------------

import pandas as pd
from sqlalchemy import create_engine, text

# ----------------------------
# Connection
# ----------------------------
engine = create_engine(
    "oracle+oracledb://",
    connect_args={"user": "raw_layer", "password": "Raw#123", "dsn": "localhost:1521/XEPDB1"},
    pool_pre_ping=True,
)


# ----------------------------
# Utility mappers / metadata
# ----------------------------
def map_table(conn, sql, key_col, val_col):
    t = pd.read_sql(text(sql), conn)
    if t.empty:
        return {}
    t.columns = [c.upper() for c in t.columns]
    return dict(zip(t[key_col.upper()].astype(str), t[val_col.upper()].astype(int)))

def _get_iot_device_pk_name(conn):
    # Prefer identity column if defined for IOT_DEVICES
    idx = pd.read_sql(
        text("SELECT COLUMN_NAME FROM USER_TAB_IDENTITY_COLS WHERE TABLE_NAME = 'IOT_DEVICES'"),
        conn
    )
    if not idx.empty and "COLUMN_NAME" in [c.upper() for c in idx.columns]:
        col = str(idx.iloc[0][idx.columns[0]]).strip()
        return col

    # Else inspect columns, prefer DEVICE_ID if present
    cols = pd.read_sql(
        text("SELECT COLUMN_NAME FROM USER_TAB_COLUMNS WHERE TABLE_NAME = 'IOT_DEVICES'"),
        conn
    )
    if cols.empty:
        raise RuntimeError("IOT_DEVICES not found or no columns visible.")
    names = [str(c).upper().strip() for c in cols["COLUMN_NAME"].tolist()]
    if "DEVICE_ID" in names:
        return "DEVICE_ID"
    return names[0]  # last resort

def _fetch_free_device_ids(conn, device_pk_col):
    # Devices INACTIVE and not linked to any CARS.DEVICE_ID
    q = f"""
        SELECT d.{device_pk_col} AS DEVICE_PK
        FROM IOT_DEVICES d
        LEFT JOIN CARS c ON c.DEVICE_ID = d.{device_pk_col}
        WHERE d.STATUS = 'INACTIVE' AND c.DEVICE_ID IS NULL
        ORDER BY d.{device_pk_col}
    """
    t = pd.read_sql(text(q), conn)
    t.columns = [c.upper().strip() for c in t.columns]
    return t["DEVICE_PK"].astype(int).tolist() if not t.empty else []

# ----------------------------
# Seeders
# ----------------------------
def seed_branches():
    data = [
        ("Casablanca HQ",   "Bd Al Massira, Maarif",           "Casablanca", "+212522000111", "casa.hq@carrental.ma"),
        ("Rabat Agdal",     "Av. de France, Agdal",            "Rabat",      "+212537000222", "rabat.agdal@carrental.ma"),
        ("Marrakech Gueliz","Av. Mohammed V, Gueliz",          "Marrakech",  "+212524000333", "marrakech.gueliz@carrental.ma"),
        ("Tanger Downtown", "Rue de la Liberté, Centre-ville", "Tanger",     "+212539000444", "tanger.dt@carrental.ma"),
        ("Agadir Plage",    "Corniche, Plage",                 "Agadir",     "+212602555666", "agadir.plage@carrental.ma"),
    ]
    df = pd.DataFrame(data, columns=["BRANCH_NAME","ADDRESS","CITY","PHONE","EMAIL"])
    with engine.begin() as conn:
        df.to_sql("BRANCHES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} branches")

def seed_managers():
    rows = [
        ("MGR101","Amina","Berrada","amina.berrada@carrental.ma","+212600100101","pwd#Casa1","Casablanca HQ"),
        ("MGR102","Karim","Saidi","karim.saidi@carrental.ma","+212600100102","pwd#Casa2","Casablanca HQ"),
        ("MGR201","Yassin","El Idrissi","yassin.elidrissi@carrental.ma","+212600200201","pwd#Rabat1","Rabat Agdal"),
        ("MGR202","Lina","Mouline","lina.mouline@carrental.ma","+212600200202","pwd#Rabat2","Rabat Agdal"),
        ("MGR301","Nadia","Zerouali","nadia.zerouali@carrental.ma","+212600300301","pwd#Mrk1","Marrakech Gueliz"),
        ("MGR302","Omar","Kabbaj","omar.kabbaj@carrental.ma","+212600300302","pwd#Mrk2","Marrakech Gueliz"),
        ("MGR401","Soukaina","Benali","soukaina.benali@carrental.ma","+212600400401","pwd#Tgr1","Tanger Downtown"),
        ("MGR402","Hicham","Alaoui","hicham.alaoui@carrental.ma","+212600400402","pwd#Tgr2","Tanger Downtown"),
        ("MGR501","Sara","El Fassi","sara.elfassi@carrental.ma","+212600500501","pwd#Agd1","Agadir Plage"),
        ("MGR502","Youssef","Boukhriss","youssef.boukhriss@carrental.ma","+212600500502","pwd#Agd2","Agadir Plage"),
    ]
    df = pd.DataFrame(rows, columns=["MANAGER_CODE","FIRST_NAME","LAST_NAME","EMAIL","PHONE","MANAGER_PASSWORD","BRANCH_NAME"])
    with engine.begin() as conn:
        bmap = map_table(conn, "SELECT BRANCH_ID, BRANCH_NAME FROM BRANCHES", "BRANCH_NAME", "BRANCH_ID")
        if not bmap:
            raise RuntimeError("Seed branches first.")
        missing = sorted(set(df.BRANCH_NAME.unique()) - set(bmap.keys()))
        if missing:
            raise RuntimeError(f"Unknown branches for managers: {missing}")
        df["BRANCH_ID"] = df["BRANCH_NAME"].map(bmap)
        ins = df[["MANAGER_CODE","FIRST_NAME","LAST_NAME","EMAIL","PHONE","MANAGER_PASSWORD","BRANCH_ID"]]
        ins.to_sql("MANAGERS", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} managers")

def seed_categories():
    rows = [
        ("Economy",  "Small city cars; fuel-efficient and affordable"),
        ("SUV",      "Sport Utility Vehicles; spacious and powerful"),
        ("Luxury",   "Premium sedans and coupes; high comfort"),
        ("Van",      "7–9 seat vehicles for families or groups"),
        ("Electric", "Fully electric vehicles; zero emissions"),
    ]
    df = pd.DataFrame(rows, columns=["CATEGORY_NAME","DESCRIPTION"])
    with engine.begin() as conn:
        df.to_sql("CAR_CATEGORIES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} car categories")

def seed_iot_devices():
    rows = [
    ("DEV001", "IMEI1000000001", "v1.0.9"),
    ("DEV002", "IMEI1000000002", "v1.1.0"),
    ("DEV003", "IMEI1000000003", "v1.1.2"),
    ("DEV004", "IMEI1000000004", "v1.1.5"),
    ("DEV005", "IMEI1000000005", "v1.2.0"),
    ("DEV006", "IMEI1000000006", "v1.2.0"),
    ("DEV007", "IMEI1000000007", "v1.2.1"),
.....
    ("DEV047", "IMEI1000000047", "v2.1.0"),
    ("DEV048", "IMEI1000000048", "v2.1.1"),
    ("DEV049", "IMEI1000000049", "v2.1.2"),
    ("DEV050", "IMEI1000000050", "v2.2.0"),
    ]

    df = pd.DataFrame(rows, columns=["DEVICE_CODE","DEVICE_IMEI","FIRMWARE_VERSION"])
    df["STATUS"] = "INACTIVE"
    df["ACTIVATED_AT"] = None
    df["LAST_SEEN_AT"] = None
    with engine.begin() as conn:
        df.to_sql("IOT_DEVICES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} IoT devices (INACTIVE)")

def seed_cars():
    rows = [
    # Casablanca HQ (15)
    ("Economy","VIN000000001","A-101-CN","Dacia","Sandero",2022,"White",12000,"AVAILABLE","Casablanca HQ"),
    ("Economy","VIN000000002","A-102-CN","Toyota","Yaris",2021,"Blue",23000,"AVAILABLE","Casablanca HQ"),
    ("Economy","VIN000000003","A-103-CN","Kia","Picanto",2023,"Red",8000,"AVAILABLE","Casablanca HQ"),
    ("SUV","VIN000000004","A-104-SV","Hyundai","Tucson",2023,"Gray",9000,"AVAILABLE","Casablanca HQ"),
    ("SUV","VIN000000005","A-105-SV","Nissan","Qashqai",2022,"Silver",11000,"AVAILABLE","Casablanca HQ"),
    ("Luxury","VIN000000006","A-106-LX","BMW","530i",2021,"Black",41000,"AVAILABLE","Casablanca HQ"),
 ....
    ("Electric","VIN000000053","E-508-EV","Tesla","Model 3",2023,"Black",7000,"AVAILABLE","Agadir Plage"),
    ("Electric","VIN000000054","E-509-EV","Renault","Zoe",2022,"Blue",8000,"AVAILABLE","Agadir Plage"),
    ("Electric","VIN000000055","E-510-EV","Peugeot","e-208",2023,"Yellow",6000,"AVAILABLE","Agadir Plage"),
    ]

    df = pd.DataFrame(rows, columns=[
        "CATEGORY_NAME","VIN","LICENSE_PLATE","MAKE","MODEL","MODEL_YEAR","COLOR","ODOMETER_KM","STATUS","BRANCH_NAME"
    ])
    with engine.begin() as conn:
        # Maps
        cmap = map_table(conn, "SELECT CATEGORY_ID, CATEGORY_NAME FROM CAR_CATEGORIES", "CATEGORY_NAME", "CATEGORY_ID")
        bmap = map_table(conn, "SELECT BRANCH_ID, BRANCH_NAME FROM BRANCHES", "BRANCH_NAME", "BRANCH_ID")
        if not cmap or not bmap:
            raise RuntimeError("Seed categories and branches first.")

        # Detect PK column of IOT_DEVICES and pull free IDs
        device_pk = _get_iot_device_pk_name(conn)
        free = _fetch_free_device_ids(conn, device_pk)

        # Validate maps
        missing_cat = sorted(set(df.CATEGORY_NAME.unique()) - set(cmap.keys()))
        missing_br  = sorted(set(df.BRANCH_NAME.unique())   - set(bmap.keys()))
        if missing_cat: raise RuntimeError(f"Unknown categories for cars: {missing_cat}")
        if missing_br:  raise RuntimeError(f"Unknown branches for cars: {missing_br}")

        # Resolve FKs + device assignment
        df["CATEGORY_ID"] = df["CATEGORY_NAME"].map(cmap)
        df["BRANCH_ID"]   = df["BRANCH_NAME"].map(bmap)
        df["DEVICE_ID"]   = [free.pop(0) if free else None for _ in range(len(df))]

        ins = df[[
            "CATEGORY_ID","DEVICE_ID","VIN","LICENSE_PLATE","MAKE","MODEL",
            "MODEL_YEAR","COLOR","ODOMETER_KM","STATUS","BRANCH_ID"
        ]]
        ins.to_sql("CARS", conn, if_exists="append", index=False)

        # 🔁 NEW: Activate any devices that got linked to cars
        assigned_ids = [int(x) for x in ins["DEVICE_ID"].dropna().unique().tolist()]
        if assigned_ids:
            id_list = ", ".join(map(str, assigned_ids))
            # Use the detected PK name for WHERE to be robust (in your schema it's DEVICE_ID)
            conn.execute(text(f"""
                UPDATE IOT_DEVICES
                   SET STATUS = 'ACTIVE',
                       ACTIVATED_AT = NVL(ACTIVATED_AT, SYSTIMESTAMP)
                 WHERE {device_pk} IN ({id_list})
            """))

    print(f"✅ Inserted {len(df)} cars with device assignment (linked devices set ACTIVE)")


# ----------------------------
# Main
# ----------------------------
def main():
    wipe_all_data()     # full reset
    seed_branches()
    seed_managers()
    seed_categories()
    seed_iot_devices()
    seed_cars()
    print("🎉 Static seed completed (fresh database).")

if __name__ == "__main__":
    main()